In [2]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import random

In [8]:
transform_default = transforms.Compose([transforms.ToTensor()])

def prepare_transforms():
    
    train_set = ImageFolder(root='cinic10/versions/1/train//', transform=transform_default)
    validate_set = ImageFolder(root='cinic10/versions/1/valid//', transform=transform_default)
    test_set = ImageFolder(root='cinic10/versions/1/test//', transform=transform_default)
        
    data_loader = DataLoader(train_set, batch_size=64, num_workers=6, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_val = DataLoader(validate_set, batch_size=128, num_workers=2, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_test = DataLoader(test_set, batch_size=128, num_workers=2, generator=torch.Generator(device='cpu'),persistent_workers=True)

    return data_loader, data_loader_val, data_loader_test





In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=4, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.linear1 = nn.Linear(3600  , 200)
        self.linear2 = nn.Linear(200, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.flatten(x, 1)  
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_2(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN_2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=4, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.linear1 = nn.Linear(32 * 7 * 7  , 100)
        self.linear2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = torch.flatten(x, 1)  
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_dropout(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN_dropout, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=4, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.2)
        self.linear1 = nn.Linear(32 * 7 * 7  , 100)
        self.linear2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = torch.flatten(x, 1)  
        x = self.dropout(x)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_2_dropout(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN_2_dropout, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=4, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.25)
        self.linear1 = nn.Linear(64 * 3 * 3  , 100)
        self.linear2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = self.dropout(x)
        x = torch.flatten(x, 1)  
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x


In [6]:
model = CNN()

In [7]:
train_set[0][0]

NameError: name 'train_set' is not defined

In [ ]:
model(torch.Tensor(1,train_set[0][0]))

In [6]:
transform = transforms.Compose([transforms.ToTensor()])

train_set = ImageFolder(root='cinic10/versions/1/train//', transform=transform)
validate_set = ImageFolder(root='cinic10/versions/1/valid//', transform=transform)
test_set = ImageFolder(root='cinic10/versions/1/test//', transform=transform)

class_names = train_set.classes
num_labels = len(class_names)

In [7]:
lr = [0.001,0.0001]
models= [CNN, CNN_2,CNN_dropout]

In [9]:
def test(model,data_loader_test, criterion,device):
    test_loss = 0
    correct = 0
    total = 0
    for i, (batch_X, batch_Y) in enumerate(data_loader_test):
        X = batch_X.to(device, non_blocking=True)
        Y = batch_Y.to(device, non_blocking=True)
        outputs = model(X)
        loss = criterion(outputs, Y)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == Y).sum().item()
        total += Y.size(0)
    avg_test_loss = test_loss / len(data_loader_test)
    test_accuracy = correct / total * 100
    print(f" Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")            
    return  avg_test_loss, test_accuracy

In [11]:
def model_train(lr, model, seed):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

    
    losses_tr = []
    accuracies_tr = []
    losses_val = []
    accuracies_val = []
    data_loader, data_loader_val, data_loader_test = prepare_transforms()
            
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
        
    num_epochs = 60
    
    prev_prev_prev_loss = float('inf')
    prev_prev_loss = float('inf')
    prev_loss = float('inf')
    curr_loss = float('inf')

    model = model()
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(num_epochs): 
        print('Entered the loop')
        model.train()
        total_loss = 0
        correct = 0
        total = 0
                
        for i, (batch_X, batch_Y) in enumerate(data_loader):
            X = batch_X.to(device, non_blocking=True)  
            Y = batch_Y.to(device, non_blocking=True)  
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, Y)
            loss.backward()
            optimizer.step()
                
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == Y).sum().item()
            total += Y.size(0)
                
    
        avg_loss = total_loss / len(data_loader)
        train_accuracy = correct / total * 100
        train_losses.append(avg_loss)
        train_accuracies.append(train_accuracy)
        print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")
            
            
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for i, (batch_X, batch_Y) in enumerate(data_loader_val):
                if i >= 140:  
                    break
                X = batch_X.to(device, non_blocking=True)
                Y = batch_Y.to(device, non_blocking=True)
                outputs = model(X)
                loss = criterion(outputs, Y)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == Y).sum().item()
                total += Y.size(0)
        avg_val_loss = val_loss / (140)
        val_accuracy = correct / total * 100
        print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")            
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)
    
        if(avg_val_loss>curr_loss and curr_loss>prev_loss and prev_loss > prev_prev_loss and prev_prev_loss> prev_prev_prev_loss):
            losses_tr.append(train_losses)
            accuracies_tr.append(train_accuracies) 
            losses_val.append(val_losses)  
            accuracies_val.append(val_accuracies)
            avg_test_loss, test_accuracy = test(model, data_loader_test, criterion, device)
            break
        prev_prev_prev_loss = prev_prev_loss  
        prev_prev_loss = prev_loss
        prev_loss = curr_loss
        curr_loss = avg_val_loss
    
        if(epoch == num_epochs - 1 ):
            losses_tr.append(train_losses)
            accuracies_tr.append(train_accuracies) 
            losses_val.append(val_losses)  
            accuracies_val.append(val_accuracies) 
            avg_test_loss, test_accuracy = test(model, data_loader_test, criterion, device)

    return model,lr,val_losses, val_accuracies, train_losses, train_accuracies, avg_test_loss, test_accuracy




In [10]:
lrate = [0.001,0.0001]
models= [CNN_dropout, CNN_2,CNN]
results = []
for lr in lrate:
    for model in models:       
        results.append(model_train(lr,model,42))

Entered the loop
Epoch 1, Average Loss: 1.6929
Epoch 1, Validation Loss: 1.5190, Accuracy: 45.08%
Entered the loop
Epoch 2, Average Loss: 1.4734
Epoch 2, Validation Loss: 1.4487, Accuracy: 46.84%
Entered the loop
Epoch 3, Average Loss: 1.3690
Epoch 3, Validation Loss: 1.3462, Accuracy: 50.77%
Entered the loop
Epoch 4, Average Loss: 1.2982
Epoch 4, Validation Loss: 1.3095, Accuracy: 52.44%
Entered the loop
Epoch 5, Average Loss: 1.2497
Epoch 5, Validation Loss: 1.2450, Accuracy: 54.93%
Entered the loop
Epoch 6, Average Loss: 1.2131
Epoch 6, Validation Loss: 1.2134, Accuracy: 55.96%
Entered the loop
Epoch 7, Average Loss: 1.1783
Epoch 7, Validation Loss: 1.1922, Accuracy: 57.25%
Entered the loop
Epoch 8, Average Loss: 1.1558
Epoch 8, Validation Loss: 1.2116, Accuracy: 56.14%
Entered the loop
Epoch 9, Average Loss: 1.1271
Epoch 9, Validation Loss: 1.1874, Accuracy: 57.57%
Entered the loop
Epoch 10, Average Loss: 1.1077
Epoch 10, Validation Loss: 1.1515, Accuracy: 58.38%
Entered the loop
E

UnboundLocalError: cannot access local variable 'avg_test_loss' where it is not associated with a value

In [11]:
results

[(CNN_dropout(
    (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (dropout): Dropout(p=0.2, inplace=False)
    (linear1): Linear(in_features=1568, out_features=100, bias=True)
    (linear2): Linear(in_features=100, out_features=10, bias=True)
  ),
  0.001,
  [1.518992062977382,
   1.4486844965389796,
   1.3462432503700257,
   1.3094680522169386,
   1.2450068478073393,
   1.2133797986166819,
   1.1922112528766904,
   1.2115562877484731,
   1.1873556396790914,
   1.1514819170747483,
   1.193324555669512,
   1.1570603102445602,
   1.1564021872622625,
   1.2162472712142127,
   1.1870364636182784,
   1.1766083968537195,
   1.1866138649838311,
   1.1479266456195287,
   1.1695971148354667,
   1.1564657224076136,

In [14]:
lrate = [0.001,0.0001]
models= [CNN_2_dropout, CNN]
results = []
for lr in lrate:
    for model in models:       
        results.append(model_train(lr,model,42))

Entered the loop
Epoch 1, Average Loss: 1.7332
Epoch 1, Validation Loss: 1.5494, Accuracy: 42.95%
Entered the loop
Epoch 2, Average Loss: 1.5068
Epoch 2, Validation Loss: 1.4610, Accuracy: 46.10%
Entered the loop
Epoch 3, Average Loss: 1.4023
Epoch 3, Validation Loss: 1.3136, Accuracy: 51.95%
Entered the loop
Epoch 4, Average Loss: 1.3263
Epoch 4, Validation Loss: 1.3305, Accuracy: 50.98%
Entered the loop
Epoch 5, Average Loss: 1.2694
Epoch 5, Validation Loss: 1.2292, Accuracy: 55.36%
Entered the loop
Epoch 6, Average Loss: 1.2270
Epoch 6, Validation Loss: 1.2025, Accuracy: 56.79%
Entered the loop
Epoch 7, Average Loss: 1.1915
Epoch 7, Validation Loss: 1.1755, Accuracy: 57.37%
Entered the loop
Epoch 8, Average Loss: 1.1656
Epoch 8, Validation Loss: 1.1764, Accuracy: 57.53%
Entered the loop
Epoch 9, Average Loss: 1.1413
Epoch 9, Validation Loss: 1.1474, Accuracy: 58.42%
Entered the loop
Epoch 10, Average Loss: 1.1190
Epoch 10, Validation Loss: 1.1413, Accuracy: 59.11%
Entered the loop
E

KeyboardInterrupt: 

In [15]:
results

[(CNN_2_dropout(
    (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (dropout): Dropout(p=0.25, inplace=False)
    (linear1): Linear(in_features=576, out_features=100, bias=True)
    (linear2): Linear(in_features=100, out_features=10, bias=True)
  ),
  0.001,
  [1.5493619407926287,
   1.4610315348420824,
   1.313620889186859,
   1.330524436065129,
   1.2291575023106167,
   1.2025145377431596,
   1.175507812840598,
   1.176432757292475,
   1.1473735579422542,
   1.1413454996688026,
   1.1510955899953843,
   1.127378837125642,
   1.127382106014